# PrivateQ

In [1]:
# Set up
import os
import sys; sys.path.append("..")
import warnings; warnings.filterwarnings("ignore")

from dotenv import load_dotenv; load_dotenv()

import ray

In [2]:
from privateq.config import ROOT_DIR

# Set up ray with credentials and start ray
ray.init(runtime_env={
    "env_vars": {
        "OPENAI_API_BASE": os.environ["OPENAI_API_BASE"],
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"], 
    },
    "working_dir": str(ROOT_DIR)
})

# Show resources
ray.cluster_resources()

2023-12-31 19:35:50,960	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2023-12-31 19:35:50,968	INFO packaging.py:530 -- Creating a file package for local directory '/Users/tiluan/git/privateq'.
2023-12-31 19:35:50,972	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_3cd9308e8687d82b.zip' (0.03MiB) to Ray cluster...
2023-12-31 19:35:50,973	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_3cd9308e8687d82b.zip'.


{'memory': 3118684571.0,
 'node:__internal_head__': 1.0,
 'CPU': 8.0,
 'node:127.0.0.1': 1.0,
 'object_store_memory': 1559342284.0}

In [7]:
# Import model parameters
from privateq.config import EMBEDDING_DIMENSIONS, MAX_CONTENT_LENGTH

print(f'EMBEDDING_DIMENSIONS: {EMBEDDING_DIMENSIONS}\nMAX_CONTENT_LENGTH: {MAX_CONTENT_LENGTH}')

EMBEDDING_DIMENSIONS: 768
MAX_CONTENT_LENGTH: 8192


In [16]:
# Shutdown Ray to clean resources
ray.shutdown()